In [1]:
import ollama
import json
from fuzzywuzzy import process
from IPython.display import display


c:\Users\Nayan\Desktop\chatInterface\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
with open("unique_names.json","r") as file:
    names=json.load(file)

with open("conciseRoom.json","r") as file:
    temp=json.load(file)
    rooms={}
    for i in temp:
        key=list(i.keys())[0]
        rooms[key]=i[key]

with open("floors.json","r") as file:
    floors=json.load(file)

print(names)

['3-D LIFT LOBBY-5', 'pantry 6-B', '5-B-1', 'Male Washroom 1', '3-B-5 GATE-2', '5-A-1 GATE-1', '3A-2F', '1A-1 Entry-1', 'STAIRS 2', 'STAIRS-1', '2-A-2 GATE-2', '6-C-LIFT LOBBY-4', '2-A-1A3', 'G-A-3-C', 'Auditorium Entry 2', 'Electrical Room-C', '3A-2J', '3B-4G', '3B-2', '5-A-2 GATE-2', 'Pantry-3-A', '1C-1B', '5-A-2 GATE-1', '4-B-1F', '2B-4H', 'AHU Room -A-L1', 'Store-3-D', '501', 'bed room', '1-B CORRIDOR-1', '2-C-3A', '1C-2D', 'Entry to D Block/Auditorium ', '404', 'Pantry-G-B', '3C-A2', '5-A-2C', '3C-A5', '2-A-2D', 'Research Park Security', 'Entry-1B-1', '1A-1B', '4-B Entry', '2-A CORRIDOR 1', 'pantry 5-A', '2C-1 Entry', '1C-2B', '3A-1B', '2-B LIFT LOBBY-2', '3C-4A', 'Pantry-3-C', '2-A-1B', 'Electronics Prototyping Lab', 'STAIRS-3', '4-B-1K', 'STAIRS-', '1C-2F', '5-C-1 GATE-1', '4-C-1E', '2B-5D', '2-C BALCONY', '3B-4E', 'Exhibition Room', '2B-3', '4-C-LIFT LOBBY-4', '2C-A5', 'Female Washroom 1', 'Office-A0', 'Pantry-G-C', 'Entry-1B-2', '3A-2H', '302', 'BEDROOM', '3B CORRIDOR 2', '3A-

In [9]:
def functionCall(query,names,rooms,floors):
    msg="Reply with JSON for the following question:"
    msg=msg+query
    genOutput= ollama.chat(
            model='functionCallQ4Temp0',
            messages=[{'role': 'user', 'content': msg}],
            stream=False,
        )['message']['content']
    functionName= json.loads(convert(genOutput))['function_name']  
    para1=None
    if('parameter_1' in json.loads(convert(genOutput)).keys()):
        para1=json.loads(convert(genOutput))['parameter_1']
    if(functionName=="roomInfo"):
        return roomInfo(para1,para1,names,5,rooms)
    elif(functionName=="floorInfo"):
        return "use this to answer the query.floor Information:"+str(floorInfo(para1,floors))
    else:
        return "Follow Up"

    
    
def floorInfo(floor,floor_list):
    if(str(floor) in floor_list.keys()):
        s="Room Information:{"
        s+=str(floor_list[str(floor)])
        s+="Room Found:True"
        s=s+"}"
        return s

def roomInfo(currName,input_string, string_list, n,room_list):
    if(input_string==None):
        input_string=currName
    
    stripped_input = input_string.replace('-', '')
    
    # Create a dictionary to map stripped strings to original strings
    stripped_to_original = {s.replace('-', ''): s for s in string_list}
    
    # Use process.extract with stripped strings
    matches = process.extract(stripped_input, stripped_to_original.keys(), limit=n)
    
    # Print matches for debugging
    print(matches)
    
    # Return original strings (with dashes) corresponding to the matches
    context="Top matches:{"
    for i in matches:
        context+=stripped_to_original[i[0]]+", "
    context+="}\n"
    context+="Room Information:{"
    if(matches[0][1]>90):
        context+=str(room_list[stripped_to_original[matches[0][0]]])
        currName=stripped_to_original[matches[0][0]]
    context+="}\n"
    if(matches[0][1]>90):
        context="Room Found:True"+context
    else:
        context="Room Found:False"+context

    print("currName:"+currName)
    print("inputString:"+input_string)
    
    return context

def convert(str):
    nstr=""
    for i in range(1,len(str)-1):
        if(str[i]=='\''):
            nstr+='\"'
        else:
            nstr+=str[i]
    return nstr

def chat(query,name_list,room_list,floor_list):
    context="{"
    context+=functionCall(query,name_list,room_list,floor_list)
    context+="context:"+"{"+context+"}"+"\n"
    context+="query:{"+query+"}\n"
    context+="}"

    print(context)
    
    stream = ollama.chat(
        model='impRag',
        messages=[{'role': 'user', 'content': context}],
        stream=True,
    )
    
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)

In [10]:
msg="Does it have whiteboard?"
#functionCall(msg,names,rooms,floors)
chat(msg,names,rooms,floors)

{Follow Upcontext:{{Follow Up}
query:{Does it have whiteboard?}
}
Based on the provided information, I do not have any specific details about the room's amenities. However, according to the floor information, there is a utility room with an electrical projector, which suggests that there might be some presentation or meeting facilities available.

To answer your query: Unfortunately, I don't have any information about whether 1A-2B has a whiteboard or not. You may want to contact the building management or the room's occupants directly for more information on this specific amenity.

In [5]:
modelfile=""
with open("chatInterface.txt","r") as file:
    for i in file:
        modelfile+=i
ollama.create(model='chatInterface', modelfile=modelfile)

{'status': 'success'}